In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from distribution_optimization_py.utils import optimal_no_bins, std_dev_estimate
from distribution_optimization_py.problem import GaussianMixtureProblem
from distribution_optimization_py.solver import iLSHADESolver
from distribution_optimization_py.solver.em import EMSolver

# best_solution = np.array([0.1, 0.1, 0.1, 0.6, 0.1, 0.05, 0.1, 1.0, 0.1, 1.0, 5, 10.0])
# best_solution = np.array([0.3, 0.1, 0.1, 0.5, 0.1, 0.05, 0.15, 1.4, 0.1, 0.4, 4.5, 10.0])
# best_solution = np.array([0.1, 0.1, 0.1, 0.1, 0.6, 1.5, 1.5, 1.5, 1.5, 10.0, 0.0, 5.0, 10.0, 50.0, 1000.0])
# best_solution = np.array(
#     [
#         0.2,
#         0.2,
#         0.1,
#         0.1,
#         0.4,
#         sigma,
#         2*sigma,
#         sigma,
#         2*sigma,
#         sigma,
#         min + 2 * sigma,
#         min + 2 * sigma + step_size,
#         expected_bin_width - step_size,
#         max - 2 * sigma - step_size,
#         max - 2 * sigma,
#     ]
# )
min = 0.0
max = 1000.0
sigma = (max - min) * 0.001 * 1.5  # 5 times larger than min std dev value
expected_bin_width = (max - min) / 10
step_size = expected_bin_width / 4
best_solution = np.array(
    [
        0.1,
        0.4,
        0.4,
        0.1,
        sigma,
        sigma,
        sigma,
        sigma,
        min + 2 * sigma,
        min + 2 * sigma + step_size,
        max - 2 * sigma - step_size,
        max - 2 * sigma,
    ]
)
problem = GaussianMixtureProblem.create_problem(best_solution)
overlap = problem.overlap_error_by_density(
    best_solution[2 * problem.nr_of_modes :],
    best_solution[problem.nr_of_modes : (2 * problem.nr_of_modes)],
    best_solution[: problem.nr_of_modes],
)
px.histogram(
    problem.data,
    nbins=problem.nr_of_bins,
    title=f"Nr of bins: {problem.nr_of_bins}, Overlap: {overlap.round(2)}",
).show()
assert (best_solution >= problem.data_lower).all() and (
    best_solution <= problem.data_upper
).all()
nr_of_bins = problem.nr_of_bins

In [2]:
from astropy.stats import bayesian_blocks

bin_edges = bayesian_blocks(problem.data, p0=0.25, fitness='events')
count, _ = np.histogram(problem.data, bins=bin_edges)
labels = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in range(len(bin_edges)-1)]

px.bar(
    x=labels,
    y=count,
    title=f"Nr of bins: {problem.nr_of_bins}, Overlap: {overlap.round(2)}",
).show()

In [7]:
nr_of_bins = 1000
problem_with_quantile_bins = GaussianMixtureProblem(
    problem.data, problem.nr_of_modes, nr_of_bins=nr_of_bins, bin_selection_method='quantiles'
)

bin_edges = problem_with_quantile_bins.breaks
count = problem_with_quantile_bins.observed_bins
labels = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in range(len(bin_edges)-1)]

px.bar(
    x=labels,
    y=count,
    title=f"Nr of bins: {problem_with_quantile_bins.nr_of_bins}, Overlap: {overlap.round(2)}",
).show()

In [7]:
std_dev_estimate(problem.data)

477.0245747453127

In [8]:
problem.data_lower, problem.data_upper

(array([ 0.03      ,  0.03      ,  0.03      ,  0.03      ,  1.00065905,
         1.00065905,  1.00065905,  1.00065905, -0.45230805, -0.45230805,
        -0.45230805, -0.45230805]),
 array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00065905e+02, 1.00065905e+02, 1.00065905e+02, 1.00065905e+02,
        1.00020674e+03, 1.00020674e+03, 1.00020674e+03, 1.00020674e+03]))

In [9]:
problem.observed_bins

array([200,   0,   0,   0,   0,   0,   0,   0,   0, 200])

In [10]:
problem.breaks

array([-4.52308045e-01,  9.96135970e+01,  1.99679502e+02,  2.99745407e+02,
        3.99811312e+02,  4.99877217e+02,  5.99943122e+02,  7.00009027e+02,
        8.00074932e+02,  9.00140837e+02,  1.00020674e+03])

In [11]:
best_solution

array([1.00e-01, 4.00e-01, 4.00e-01, 1.00e-01, 1.50e+00, 1.50e+00,
       1.50e+00, 1.50e+00, 3.00e+00, 2.80e+01, 9.72e+02, 9.97e+02])

In [12]:
nr_of_bins_to_solution = {}
for nr_of_bins in [nr_of_bins, 10, 20, 30, 40, 50, 60, 75, 100]:
    problem = GaussianMixtureProblem(problem.data, problem.nr_of_modes, nr_of_bins=nr_of_bins)
    solver = iLSHADESolver()
    solution = solver(problem, 10000)
    nr_of_bins_to_solution[nr_of_bins] = solution.genome
    print(nr_of_bins, solution.fitness)

10 0.0
10 0.020305572589720877
20 0.0
30 0.0
40 0.0
50 0.0
60 0.0
75 0.57974460555443
100 0.618358124809059


In [13]:
nr_of_bins_to_quantile_solution = {}
for nr_of_bins in [nr_of_bins, 10, 20, 30, 40, 50, 60, 75, 100]:
    problem = GaussianMixtureProblem(problem.data, problem.nr_of_modes, nr_of_bins=nr_of_bins, bin_selection_method="quantiles")
    solver = iLSHADESolver()
    solution = solver(problem, 10000)
    nr_of_bins_to_quantile_solution[nr_of_bins] = solution.genome
    print(nr_of_bins, solution.fitness)

100 0.13325541825190962
10 0.002362997126943627
20 2.1931796209615815
30 1.1816065513164222
40 0.027919070935766746
50 0.032143112612713284
60 0.0954752372795647
75 0.5944655529307773
100 0.1032098874495921


In [15]:
bayesian_block_problem = GaussianMixtureProblem(problem.data, problem.nr_of_modes, bin_selection_method="bayesian_blocks")
solver = iLSHADESolver()
bayesian_block_solution = solver(bayesian_block_problem, 10000)

In [21]:
from distribution_optimization_py.gaussian_mixture import compare_solutions_plotly

solutions = list(nr_of_bins_to_solution.values())
labels = [f"{nr_of_bins}" for nr_of_bins in nr_of_bins_to_solution.keys()]
quantile_solutions = list(nr_of_bins_to_quantile_solution.values())
quantile_labels = [f"{nr_of_bins} quantiles" for nr_of_bins in nr_of_bins_to_quantile_solution.keys()]

em_solver = EMSolver()
em_solution = em_solver(problem, 1000, 42)

compare_solutions_plotly(
    problem.data,
    problem.nr_of_modes,
    solutions + quantile_solutions + [best_solution, em_solution.genome, bayesian_block_solution.genome],
    labels + quantile_labels + ["Best", "EM", "BayesianBlock"],
    bins=500,
)

In [20]:
problem.fix(solutions[0]), best_solution

(array([2.99783465e-02, 4.72766794e-01, 4.65400922e-01, 3.18539375e-02,
        7.00503803e+01, 9.06351909e+00, 3.27757992e+00, 5.02278006e+01,
        9.50265663e+00, 3.96496676e+01, 9.54669754e+02, 1.00020674e+03]),
 array([1.00e-01, 4.00e-01, 4.00e-01, 1.00e-01, 1.50e+00, 1.50e+00,
        1.50e+00, 1.50e+00, 3.00e+00, 2.80e+01, 9.72e+02, 9.97e+02]))

In [103]:
for nr_of_modes in [30]:
    random_state = np.random.RandomState(seed=1)
    means = np.linspace(0, 1, nr_of_modes, endpoint=False)
    std = (means[1] - means[0]) / 4
    random_data = np.concatenate(
        [random_state.normal(mean, std, 100) for mean in means]
    )
    best_solution = np.array(
        [1 / nr_of_modes] * nr_of_modes + [std] * nr_of_modes + [mean for mean in means]
    )
    px.histogram(random_data, nbins=optimal_no_bins(random_data)).show()
    problem = GaussianMixtureProblem(random_data, nr_of_modes)
    solver = iLSHADESolver()
    em_solver = EMSolver()
    solution = solver(problem, 10000, 42)
    em_solution = em_solver(problem, 10000, 42)
    compare_solutions_plotly(
        random_data,
        nr_of_modes,
        solutions=[solution.genome, em_solution.genome, best_solution],
        labels=["DO", "EM", "Best"],
        bins=optimal_no_bins(random_data),
    )
    problem = GaussianMixtureProblem(
        random_data, nr_of_modes, nr_of_bins=8 * nr_of_modes
    )
    solver = iLSHADESolver()
    em_solver = EMSolver()
    solution = solver(problem, 10000, 42)
    em_solution = em_solver(problem, 10000, 42)
    compare_solutions_plotly(
        random_data,
        nr_of_modes,
        solutions=[solution.genome, em_solution.genome, best_solution],
        labels=["DO", "EM", "Best"],
        bins=8 * nr_of_modes,
    )

In [21]:
nr_of_bins_to_solution = {}
for nr_of_bins in [10, 20, 30, 40, 50, 60, 75, 100]:
    problem = GaussianMixtureProblem(random_data, nr_of_modes, nr_of_bins=nr_of_bins)
    solver = iLSHADESolver()
    solution = solver(problem, 10000)
    nr_of_bins_to_solution[nr_of_bins] = solution.genome
    print(nr_of_bins, solution.fitness)

10 0.0
20 0.0
30 0.017869456508108176
40 0.10393035637761844
50 0.0423120010237055
60 0.06984142182052809
75 0.07486071053541325
100 0.09250678442011988


In [24]:
from distribution_optimization_py.gaussian_mixture import compare_solutions_plotly

best_solution = np.array([0.5] * 11 + [0.5, 1.5, 1.0, 0.15, 0.05, 1.0, 0.05, 0.15, 1.0, 1.5, 0.5] + [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0])

solutions = list(nr_of_bins_to_solution.values())
labels = [f"{nr_of_bins}" for nr_of_bins in nr_of_bins_to_solution.keys()]

compare_solutions_plotly(
    random_data,
    11,
    solutions + [best_solution],
    labels + ["Best"],
    bins=100,
)

In [15]:
from distribution_optimization_py.gaussian_mixture import compare_solutions_plotly

best_solution = np.array(
    [
        0.2,
        0.2,
        0.1,
        0.1,
        0.2,
        0.2,
        0.5,
        0.5,
        0.5,
        0.15,
        0.75,
        0.25,
        -1.0,
        -0.5,
        0.0,
        0.25,
        0.5,
        1.0,
    ]
)

problem = GaussianMixtureProblem.create_problem(best_solution)
px.histogram(problem.data, nbins=problem.nr_of_bins).show()
solver = iLSHADESolver()
em_solver = EMSolver()
solution = solver(problem, 10000, 42)
em_solution = em_solver(problem, 10000, 42)
compare_solutions_plotly(
    problem.data,
    problem.nr_of_modes,
    solutions=[solution.genome, em_solution.genome, best_solution],
    labels=["DO", "EM", "Best"],
    bins=8 * problem.nr_of_modes,
)
compare_solutions_plotly(
    problem.data,
    problem.nr_of_modes,
    solutions=[solution.genome, em_solution.genome, best_solution],
    labels=["DO", "EM", "Best"],
    bins=problem.nr_of_bins,
)
problem = GaussianMixtureProblem(
    problem.data, problem.nr_of_modes, nr_of_bins=8 * problem.nr_of_modes
)
solver = iLSHADESolver()
em_solver = EMSolver()
solution = solver(problem, 10000, 42)
em_solution = em_solver(problem, 10000, 42)
compare_solutions_plotly(
    problem.data,
    problem.nr_of_modes,
    solutions=[solution.genome, em_solution.genome, best_solution],
    labels=["DO", "EM", "Best"],
    bins=8 * problem.nr_of_modes,
)